<a href="https://colab.research.google.com/github/Ccode104/Artificial-Intelligence/blob/main/DPLL_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys

class SatSolver(object):

    def __init__(self, cnf):
        self.cnf = cnf
        self.assignment = {}
        self.trail = []
        self.level = 0

    def solve(self):
        if self.propagate() == "Conflict":
            return "UNSAT"

        while True:
            literal = self.decide()
            if literal is None:
                return "SAT"

            self.trail.append((literal, self.level))
            self.level += 1

            result = self.propagate()
            if result == "Conflict":
                backtrack_level = self.analyze_conflict()
                if backtrack_level < 0:
                    return "UNSAT"
                self.backtrack(backtrack_level)

    def propagate(self):
        while True:
            unit_clauses = [clause for clause in self.cnf if len(clause) == 1]
            if not unit_clauses:
                break

            unit_literal = unit_clauses[0][0]
            if self.assign(unit_literal) == "Conflict":
                return "Conflict"

            self.cnf = self.simplify_cnf(unit_literal)

        return "OK"

    def assign(self, literal):
        variable = abs(literal)
        if variable in self.assignment:
            if self.assignment[variable] != literal:
                return "Conflict"
        else:
            self.assignment[variable] = literal
        return "OK"


    def simplify_cnf(self, literal):
        new_cnf = []
        for clause in self.cnf:
            if literal in clause:
                continue
            new_clause = [l for l in clause if l != -literal]
            if not new_clause:
                return "Conflict" # Conflict detected
            new_cnf.append(new_clause)
        return new_cnf


    def decide(self):
        for clause in self.cnf:
            for literal in clause:
                if abs(literal) not in self.assignment:
                    return literal
        return None

    def analyze_conflict(self):
        # Simplified conflict analysis (backtrack to the highest decision level in the conflict clause)
        conflict_clause = [literal for clause in self.cnf for literal in clause if self.is_conflict_clause(clause)]
        if not conflict_clause:
            return -1 # No conflict clause found, UNSAT

        max_level = -1
        for literal in conflict_clause:
            variable = abs(literal)
            if variable in self.assignment:
                for assigned_literal, level in self.trail:
                    if abs(assigned_literal) == variable:
                        max_level = max(max_level, level)
                        break
        return max_level

    def is_conflict_clause(self, clause):
        return all(abs(literal) in self.assignment and self.assignment[abs(literal)] != literal for literal in clause)


    def backtrack(self, level):
        new_trail = []
        new_assignment = {}
        for literal, assigned_level in self.trail:
            if assigned_level < level:
                new_trail.append((literal, assigned_level))
                new_assignment[abs(literal)] = literal
        self.trail = new_trail
        self.assignment = new_assignment
        self.level = level

        # Rebuild the CNF based on the new assignment
        self.cnf = self.build_cnf_from_assignment()


    def build_cnf_from_assignment(self):
        original_cnf = self.get_original_cnf() # You would need to store or regenerate the original CNF
        new_cnf = []
        for clause in original_cnf:
            is_satisfied = False
            for literal in clause:
                variable = abs(literal)
                if variable in self.assignment:
                    if self.assignment[variable] == literal:
                        is_satisfied = True
                        break
            if not is_satisfied:
                new_clause = [l for l in clause if abs(l) not in self.assignment]
                new_cnf.append(new_clause)
        return new_cnf

    def get_original_cnf(self):
      # In a real implementation, you would pass or store the original CNF
      # For this example, we will assume the original CNF is stored in self._original_cnf
      if hasattr(self, '_original_cnf'):
          return self._original_cnf
      else:
          # This is a placeholder. You need to initialize _original_cnf in the constructor
          # or pass it to the solve method.
          print("Warning: Original CNF not stored. Using a placeholder.")
          return []


# Example usage:
# CNF in Dimacs format (list of lists of integers)
# [[1, 2], [-1, 2], [1, -2], [-1, -2]]  # Example of an UNSAT formula
# [[1, 2], [-1, 3], [-2, -3]] # Example of a SAT formula

# Example 1: UNSAT formula
cnf_unsat = [[1, 2], [-1, 2], [1, -2], [-1, -2]]
solver_unsat = SatSolver(cnf_unsat)
result_unsat = solver_unsat.solve()
print(f"Formula: {cnf_unsat}")
print(f"Result: {result_unsat}")
print(f"Assignment: {solver_unsat.assignment}")

# Example 2: SAT formula
cnf_sat = [[1, 2], [-1, 3], [-2, -3]]
solver_sat = SatSolver(cnf_sat)
result_sat = solver_sat.solve()
print(f"Formula: {cnf_sat}")
print(f"Result: {result_sat}")
print(f"Assignment: {solver_sat.assignment}")